<a href="https://colab.research.google.com/github/BossRobin/NLP/blob/master/NER4E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#! coding=utf-8
# FIRST=======================统计的方法
import pandas as pd
import numpy as np
# latin1是Mysql的默认字符集
data = pd.read_csv("drive/My Drive/ner_dataset.csv", encoding="latin1")
# ffill是向下填充，即NaN值填充为它上面的值，bfill是向上填充。
data = data.fillna(method="ffill")
print(data.head(10))

    Sentence #           Word  POS    Tag
0  Sentence: 1      Thousands  NNS      O
1  Sentence: 1             of   IN      O
2  Sentence: 1  demonstrators  NNS      O
3  Sentence: 1           have  VBP      O
4  Sentence: 1        marched  VBN      O
5  Sentence: 1        through   IN      O
6  Sentence: 1         London  NNP  B-geo
7  Sentence: 1             to   TO      O
8  Sentence: 1        protest   VB      O
9  Sentence: 1            the   DT      O


In [ ]:
words = list(set(data["Word"].values))
n_words = len(words)

print(n_words, "\n")

35178 



In [ ]:
# 预处理数据，生成训练集和标签
class SentenceGetter(object):
  def __init__(self, data):
    self.data = data
    self.n_sent = 1
    self.empty = False
  
  def get_next(self):
    try:
      s = self.data[self.data["Sentence #"] == "Sentence: {}".format(self.n_sent)]
      self.n_sent += 1
      return s["Word"].values.tolist(), s["POS"].values.tolist(), s["Tag"].values.tolist()
    except:
      self.empty = True
      return None, None, None

getter = SentenceGetter(data)
sent, pos, tag = getter.get_next()
print(sent, pos, tag)

['Thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.'] ['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP', 'TO', 'VB', 'DT', 'NN', 'IN', 'NNP', 'CC', 'VB', 'DT', 'NN', 'IN', 'JJ', 'NNS', 'IN', 'DT', 'NN', '.'] ['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class MemoryTagger(BaseEstimator, TransformerMixin):
  # fit即train
  def fit(self, X, y):
    voc = {}
    self.tags = []
    for w, t in zip(X, y):
      # self.tags统计标签的种类
      if t not in self.tags:
        self.tags.append(t)
      # voc统计训练集中的每一个词对应的标签出现的次数
      if w not in voc:
        voc[w] = {}
      if t not in voc[w]:
        voc[w][t] = 0
      voc[w][t] += 1
    # self.memory统计的是字典中每一个词对应的出现频次最大的标签
    self.memory = {}
    for k, d in voc.items():
      self.memory[k] = max(d, key=d.get)
  def predict(self, X, y=None):
    # 返回词汇x对应的标签，如果memory中没有，返回0.
    return [self.memory.get(x, "O") for x in X]
# 测试一句
tagger = MemoryTagger()
tagger.fit(sent, tag)
print(tagger.predict(sent))

['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']


In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
# 得到所有的词汇
words = data["Word"].values.tolist()
# 得到所有词汇对应的标签
tags = data["Tag"].values.tolist()
# 因为MemoryTagger继承了(BaseEstimator, TransformerMixin)，所以可以直接使用sklearn的cross_val_predict进行交叉验证。
pred = cross_val_predict(estimator=MemoryTagger(), X=words, y=tags, cv=5)
report = classification_report(y_pred=pred, y_true=tags)
print(report, "\n")

              precision    recall  f1-score   support

       B-art       0.20      0.05      0.09       402
       B-eve       0.54      0.25      0.34       308
       B-geo       0.78      0.85      0.81     37644
       B-gpe       0.94      0.93      0.94     15870
       B-nat       0.42      0.28      0.33       201
       B-org       0.67      0.49      0.56     20143
       B-per       0.78      0.65      0.71     16990
       B-tim       0.87      0.77      0.82     20333
       I-art       0.04      0.01      0.01       297
       I-eve       0.39      0.12      0.18       253
       I-geo       0.73      0.58      0.65      7414
       I-gpe       0.62      0.45      0.52       198
       I-nat       0.00      0.00      0.00        51
       I-org       0.69      0.53      0.60     16784
       I-per       0.73      0.65      0.69     17251
       I-tim       0.58      0.13      0.21      6528
           O       0.97      0.99      0.98    887908

    accuracy              

In [ ]:
# SECOND=======================机器学习方法
# 使用随机森林分类器
from sklearn.ensemble import RandomForestClassifier
# 简单的构造特征的函数
def feature_map(word):
  return np.array([word.istitle(), word.islower(), word.isupper(), len(word), word.isdigit(), word.isalpha()])
# 得到每个词的特征
words = [feature_map(w) for w in data["Word"].values.tolist()]
# 开始训练
pred = cross_val_predict(RandomForestClassifier(n_estimators=20), X=words, y=tags, cv=5)
report = classification_report(y_pred=pred, y_true=tags)
print(report)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00       402
       B-eve       0.00      0.00      0.00       308
       B-geo       0.26      0.79      0.40     37644
       B-gpe       0.26      0.06      0.09     15870
       B-nat       0.00      0.00      0.00       201
       B-org       0.65      0.17      0.27     20143
       B-per       0.97      0.20      0.33     16990
       B-tim       0.29      0.32      0.30     20333
       I-art       0.00      0.00      0.00       297
       I-eve       0.00      0.00      0.00       253
       I-geo       0.00      0.00      0.00      7414
       I-gpe       0.00      0.00      0.00       198
       I-nat       0.00      0.00      0.00        51
       I-org       0.36      0.03      0.06     16784
       I-per       0.47      0.02      0.04     17251
       I-tim       0.50      0.06      0.11      6528
           O       0.97      0.98      0.97    887908

    accuracy              

In [ ]:
# -*- coding: utf-8 -*-
# THIRD====================CRF
import pandas as pd
import numpy as np
data = pd.read_csv("drive/My Drive/ner_dataset.csv", encoding="latin1")
data = data.fillna(method="ffill")

words = list(set(data["Word"].values))
n_words = len(words)

In [ ]:
"""我们的数据中共有47959个句子，其中包含了35178个单词。 
现在来构建一个输出句子的构造器。
和第一种方法类似
"""
class SentenceGetter(object):
  def __init__(self, data):
    self.n_sent = 0
    self.data = data
    self.empty = False
    agg_func = lambda s : [(w,p,t) for w,p,t in zip(s["Word"].values.tolist(),s["POS"].values.tolist(),s["Tag"].values.tolist())]
    self.grouped = self.data.groupby("Sentence #").apply(agg_func)
    self.sentences = [s for s in self.grouped]
  def get_next(self):
    try:
      s = self.sentences[self.n_sent]
      self.n_sent += 1
      return s
    except:
      self.empty = True
      return None
getter = SentenceGetter(data)
sent = getter.get_next()
print(sent)

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


In [ ]:
# 每个句子都是词的三元组构成
sentences = getter.sentences

In [ ]:
# 将每个词转化为特征向量
def word2features(sent, i):
  # sent[i][0]是第i个单词的token值，sent[i][1]是第i个单词的pos值
  word = sent[i][0]
  postag = sent[i][1]
  # 针对单个单词的特征函数
  features = {
      "bias": 1.0,
      "word.lower()": word.lower(),
      "word[-3:]": word[-3:],
      "word[2:]": word[2:],
      "word.isupper()": word.isupper(),
      "word.istitle()": word.istitle(),
      "word.isdigit()": word.isdigit(),
      "postag": postag,
      "postag[:2]": postag[:2]
  }
  # 针对有前缀单词的单词的特征函数
  if i > 0:
    word1 = sent[i-1][0]
    postag1 = sent[i-1][1]
    features.update({
        "-1:word.isupper()": word1.isupper(),
        "-1:word.istitle()": word1.istitle(),
        "-1:word.isdigit()": word1.isdigit(),
        "-1:postag": postag1,
        "-1:postag[:2]": postag1[:2]
    })
  else:
    features["BOS"] = True
  # 针对有后缀单词的单词的特征函数
  if i < len(sent) - 1:
    word1 = sent[i+1][0]
    postag1 = sent[i+1][1]
    features.update({
      "+1:word.isupper()": word1.isupper(),
      "+1:word.istitle()": word1.istitle(),
      "+1:word.isdigit()": word1.isdigit(),
      "+1:postag": postag1,
      "+1:postag[:2]": postag1[:2]
    })
  else:
    features["EOS"] = True
  return features
# 将一个句子转化为特征向量
def sent2features(sent):
  return [word2features(sent, i) for i in range(len(sent))]
# 提取句子中的每个单词对应的标签
def sent2labels(sent):
  return [label for _, _, label in sent]
# 提取句子中单词的token
def sent2tokens(sent):
  return [token for token, _, _ in sent]

print(sent2features(sentences[0]))

print(sent2labels(sentences[0]))

print(sent2tokens(sentences[0]))

[{'bias': 1.0, 'word.lower()': 'thousands', 'word[-3:]': 'nds', 'word[2:]': 'ousands', 'word.isupper()': False, 'word.istitle()': True, 'word.isdigit()': False, 'postag': 'NNS', 'postag[:2]': 'NN', 'BOS': True, '+1:word.isupper()': False, '+1:word.istitle()': False, '+1:word.isdigit()': False, '+1:postag': 'IN', '+1:postag[:2]': 'IN'}, {'bias': 1.0, 'word.lower()': 'of', 'word[-3:]': 'of', 'word[2:]': '', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'IN', 'postag[:2]': 'IN', '-1:word.isupper()': False, '-1:word.istitle()': True, '-1:word.isdigit()': False, '-1:postag': 'NNS', '-1:postag[:2]': 'NN', '+1:word.isupper()': False, '+1:word.istitle()': False, '+1:word.isdigit()': False, '+1:postag': 'NNS', '+1:postag[:2]': 'NN'}, {'bias': 1.0, 'word.lower()': 'demonstrators', 'word[-3:]': 'ors', 'word[2:]': 'monstrators', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'NNS', 'postag[:2]': 'NN', '-1:word.isupper()':

In [ ]:
# 得到所有句子的特征向量和标签
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [ ]:
from sklearn_crfsuite import CRF
# 构建CRF分类器
crf = CRF(
    algorithm = "lbfgs",# lbfgs为无约束优化算法
    c1 = 0.1, # L1正则化的力度
    c2 = 0.1, # L2正则化的力度
    max_iterations = 100,
    all_possible_transitions = False
)

from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report

pred = cross_val_predict(estimator=crf,X=X,y=y,cv=5)

report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


              precision    recall  f1-score   support

       B-art       0.33      0.12      0.17       402
       B-eve       0.53      0.37      0.43       308
       B-geo       0.84      0.91      0.87     37644
       B-gpe       0.97      0.94      0.96     15870
       B-nat       0.74      0.37      0.49       201
       B-org       0.79      0.70      0.74     20143
       B-per       0.84      0.80      0.82     16990
       B-tim       0.93      0.81      0.87     20333
       I-art       0.14      0.04      0.07       297
       I-eve       0.36      0.24      0.29       253
       I-geo       0.80      0.78      0.79      7414
       I-gpe       0.92      0.54      0.68       198
       I-nat       0.62      0.29      0.40        51
       I-org       0.80      0.79      0.79     16784
       I-per       0.83      0.90      0.86     17251
       I-tim       0.82      0.73      0.77      6528
           O       0.99      0.99      0.99    887908

    accuracy              

In [ ]:
# !pip install eli5
"""可视化tags间的转移分数以及各features的重要性"""
crf.fit(X, y)
import eli5
eli5.show_weights(crf, top=30)


In [ ]:
crf = CRF(
    algorithm='lbfgs',
    c1=10, # 通过增加c1增加L1正则化的力度，使features变稀疏
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=False
)
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)
report = classification_report(y_pred=pred, y_true=y)
print(report)

crf.fit(X, y)


eli5.show_weights(crf, top=30)

"""增加L1正则后，features减少，但是模型效果仍旧不错"""

In [1]:
# -*- coding: utf-8 -*-
# FORTH================BERT
import pandas as pd
import numpy as np
data = pd.read_csv("drive/My Drive/ner_dataset.csv", encoding="latin1").fillna(method="ffill")

In [2]:
"""构建SentenceGetter"""
class SentenceGetter(object):
  def __init__(self, data):
    self.n_sent = 1
    self.data = data
    self.empty = False
    agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),s["POS"].values.tolist(),s["Tag"].values.tolist())]
    self.grouped = self.data.groupby("Sentence #").apply(agg_func)
    self.sentences = [s for s in self.grouped]
  
  def get_next(self):
    try:
      s = self.grouped["Sentence: {}".format(self.n_sent)]
      self.n_sent += 1
      return s
    except:
      self.empty = True
      return None

getter = SentenceGetter(data)

sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
print(sentences[0])

labels = [[s[2] for s in sent] for sent in getter.sentences]
print(labels[0])

Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']


In [3]:
"""构建tag词典"""
tags_vals = list(set(data["Tag"].values))
tag2idx = {t: i for i, t in enumerate(tags_vals)}

In [ ]:

# !pip install pytorch_pretrained_bert
"""导入相关库"""
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

In [5]:
"""设置基本参数"""
max_len = 60
batch_size = 32

"""设置device"""

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [16]:
"""tokenize处理"""
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print(tokenized_texts[0])

"""将输入转化为id 并且 截长补短"""
# 截断补全操作。对于大于max_len的句子进行截断，不足的则补全。truncating="post"意思是后面截断，padding="post"意思是后面补全。
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids[0])

tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=max_len, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")
print(tags[0])

['thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'london', 'to', 'protest', 'the', 'war', 'in', 'iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'british', 'troops', 'from', 'that', 'country', '.']
[ 5190  1997 28337  2031  9847  2083  2414  2000  6186  1996  2162  1999
  5712  1998  5157  1996 10534  1997  2329  3629  2013  2008  2406  1012
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0]
[7 7 7 7 7 7 8 7 7 7 7 7 8 7 7 7 7 7 0 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]


In [17]:
"""准备mask_attention"""
# padding的位置不用mask，设置为0，其余为1
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [18]:
"""将数据进行划分"""
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, random_state=2019, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=2019, test_size=0.1)

In [19]:
"""将数据转化为tensor的形式"""
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [20]:
"""定义dataloader,在训练阶段shuffle数据，预测阶段不需要shuffle"""
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data) #预测阶段需要shuffle
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)  #测试阶段不需要shuffle
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_size)


In [ ]:
"""**开始训练过程**"""
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))

model.cuda()

In [22]:
"""定义optimizer(分为是否调整全部参数两种情况)"""
FULL_FINETUNING = True
if FULL_FINETUNING:
    # 列举模型的全部参数名
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta'] # 不需要正则化的参数
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = BertAdam(optimizer_grouped_parameters, lr=3e-5)

t_total value of -1 results in schedule not being applied


In [23]:
# !pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=128cc05feb2c3b713c36c10546c76a5653d856fa7f73dc8afb180e08f600eeab
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [24]:
"""定义评估accuracy的函数

f1: https://blog.csdn.net/qq_37466121/article/details/87719044
"""
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [27]:
from tqdm import tqdm, trange

In [29]:
"""开始微调过程，建议4个左右epochs"""
epochs = 5
max_grad_norm = 1.0

for _ in range(epochs): # trange有可视化功能
  # 训练过程
  model.train()
  tr_loss = 0
  nb_tr_steps = 0
  for step, batch in enumerate(train_dataloader):
    # 将batch设置为gpu模式
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    # 前向过程
    loss = model(b_input_ids, token_type_ids=None,
                  attention_mask=b_input_mask, labels=b_labels)
    # 后向过程
    loss.backward()
    # 损失
    tr_loss += loss.item()
    nb_tr_steps += 1
    # 梯度裁剪
    torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
    # 更新参数
    optimizer.step()
    model.zero_grad()
  #打印每个epoch的损失
  print("Train loss: {}".format(tr_loss/nb_tr_steps))
  # 验证过程
  model.eval()
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps = 0
  predictions , true_labels = [], []
  for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    # no_grad是不需要反向传播梯度
    with torch.no_grad():
      # 因为误差不反向传播，所以可以使用验证数据集当成训练集输入模型得到验证集上的误差。
      tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
      # 得到真正的模型预测的验证集的结果，以softmax形式输出
      logits = model(b_input_ids, token_type_ids=None,
                      attention_mask=b_input_mask)
    logits = logits.detach().cpu().numpy()#detach的方法，将variable参数从网络中隔离开，不参与参数更新
    label_ids = b_labels.to('cpu').numpy()

    # print("label_ids", label_ids)
    # print("np.argmax(logits, axis=2)", np.argmax(logits, axis=2))
    # predictions保存了每个句子中每个词预测的tag结果
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.append(label_ids)
    # 计算accuracy 和 loss
    # 当前batch的验证集的准确率
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1
  # 打印信息
  print("Validation loss: {}".format(eval_loss/nb_eval_steps))
  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
  pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
  valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
  print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))#传入的是具体的tag


Train loss: 0.15560755134457302
Validation loss: 0.1590871246655782
Validation Accuracy: 0.9327579365079361
F1-Score: 0.48563854670201156
Train loss: 0.11980487075423735
Validation loss: 0.14978278296689193
Validation Accuracy: 0.9338953373015872
F1-Score: 0.5020317130549021


KeyboardInterrupt: ignored